In [4]:
!pip install pandas
!pip install numpy
!pip install matplotlib
! pip install nltk

In [6]:
import pandas as pd

In [ ]:
import os
naming_convention = "ML_Dataset_email_classifier_"
folder_path = "datasets"

# Get the list of all files in the folder
all_files = os.listdir(folder_path)
dataset_files = [os.path.join(folder_path, file) for file in all_files]
dataset_files.sort(key=os.path.getctime, reverse=True)

In [ ]:
# # initialize datasets here
# dataset = "Internship Tracker - ML_Dataset_email_classifier.csv"
# dataset_kavya = "ML_Dataset_email_classifier_kavya.csv"
# dataset_hamza = "ML_Dataset_email_classifier_hamza.csv"
# dataset_ary = "ML_Dataset_email_classifier_new_ary.csv"
# dataset_sankaet = "ML_Dataset_email_classifier_sankaet.csv"

In [134]:
def read_datasets(dataset_files):
    datasets = {}
    for file in dataset_files:
        name = "dataset_" + file.split("/")[-1].split(".")[0].split("_")[-1]
        datasets[name] = pd.read_csv(file)
    return datasets
datasets = read_datasets(dataset_files)
print(datasets.keys())

dict_keys(['dataset_sankaet', 'dataset_original', 'dataset_hamza', 'dataset_ary', 'dataset_kavya'])


In [ ]:
# Load the dataset
# create empty dataframe
data = pd.DataFrame() # to reset the data
for index, (name, dataset) in enumerate(datasets.items()):
    if name == "dataset_original" or index == 0:
        #skip over the original dataset
        continue
    data = pd.concat([data, datasets[name]], ignore_index=True)

In [157]:
# Check the first few rows of your dataset
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832 entries, 0 to 831
Data columns (total 3 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   email subject                            832 non-null    object 
 1   email body                               831 non-null    object 
 2   binary classification (expected output)  0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 19.6+ KB


,binary classification (expected output)
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [158]:
# create a random sample of 100 rows of the first dataset and append it to the current dataset
# the first dataset is the most recent dataset which is first file
first_dataset_name = list(datasets.keys())[0]
first_dataset = datasets[first_dataset_name]
sample = first_dataset.sample(n=100)
data = pd.concat([data, sample], ignore_index=True)

In [159]:
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932 entries, 0 to 931
Data columns (total 3 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   email subject                            932 non-null    object 
 1   email body                               931 non-null    object 
 2   binary classification (expected output)  0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 22.0+ KB


,binary classification (expected output)
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [160]:
# Check for missing values
data.isnull().sum()

# Drop rows with missing values (if necessary)
data.dropna(subset=['email subject', 'email body'], inplace=True)

#drop rows that are duplicates
data.drop_duplicates(subset=['email subject', 'email body'], inplace=True)

# Optionally, inspect cleaned data
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 0 to 931
Data columns (total 3 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   email subject                            905 non-null    object 
 1   email body                               905 non-null    object 
 2   binary classification (expected output)  0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 28.3+ KB


,binary classification (expected output)
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Define a function to clean the text
def clean_text(text):
    # Tokenize and lower case the text
    text = text.lower()
    tokens = word_tokenize(text)
    
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    
    return " ".join(cleaned_tokens)

clean_data = data.copy()

# Apply the cleaning function to both 'email subject' and 'email body'
clean_data['cleaned email subject'] = data['email subject'].apply(clean_text)
clean_data['cleaned email body'] = data['email body'].apply(clean_text)


In [161]:
import os
def save_accuracy_score(accuracy, file_path='accuracy_scores.csv'):
    # Check if the file exists and is not empty
    if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
        # Create a new DataFrame with headers if the file does not exist or is empty
        scores_df = pd.DataFrame(columns=['Iteration', 'Accuracy'])
    else:
        # Load existing scores if the file exists and is not empty
        scores_df = pd.read_csv(file_path)

    # Append the new accuracy score
    new_score = {'Iteration': len(scores_df) + 1, 'Accuracy': accuracy}
    scores_df = pd.concat([scores_df, pd.DataFrame([new_score])], ignore_index=True)

    # Save the updated scores to the file
    scores_df.to_csv(file_path, index=False)

In [162]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [169]:
# Combine the subject and body into a single text feature
clean_data['combined_text'] = clean_data['cleaned email subject'] + " " + clean_data['cleaned email body']

# Define your features (X) and labels (y)
X = clean_data['combined_text']
y = data['binary classification (expected output)']  # This should be a column with binary labels (1 for job applications, 0 otherwise)

In [171]:
# Encode labels into binary format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [180]:
from sklearn.linear_model import LogisticRegression

# Separate labeled and unlabeled data
labeled_data = clean_data[data['binary classification (expected output)'].notnull()]
before_labeled_data = len(labeled_data)

unlabeled_data = clean_data[data['binary classification (expected output)'].isnull()]
before_unlabeled_data = len(unlabeled_data)

print("Number of labeled data points:", before_labeled_data)
print("Number of unlabeled data points:", before_unlabeled_data)
# check whether unlabeled data is empty
if(unlabeled_data.empty):
    print("All the data is labeled")

/var/folders/kq/cs03j9553ng9l_hc6_yh2qyw0000gn/T/ipykernel_7521/407472927.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  labeled_data = clean_data[data['binary classification (expected output)'].notnull()]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [173]:
def correct_prediction(incorrect_index, run_function):
    if incorrect_index is None or not run_function:
        return
    try:
        # Correct the prediction for the given index
        unlabeled_data.at[incorrect_index, 'predictions'] = 1
        # print("This works")
    except:
        print("Index out of range")
        return

In [174]:
print(unlabeled_data)

                                         email subject  \
0                 Thank you for applying to Salesforce   
1    [Action Requested by 1/10]: Your Wayfair Softw...   
2    Your Application for Software Engineering Inte...   
3    Update on your application: We're moving forwa...   
4                       BlackRock | Application update   
..                                                 ...   
927  “intern”: RippleMatch - Data Analyst Intern, a...   
928  Thank you for applying to Twitch, please compl...   
929               Update on your Application to Clever   
930            Thank you for your application to Figma   
931  Application Update from Unity - Software Engin...   

                                            email body  \
0    Hello, Thank you for considering a career with...   
1    Hi there, Thank you for your interest in the S...   
2    Dear Hamza, Thank you again for taking the tim...   
3    Dear Hamza , Thank you again for taking the ti...   
4    Hi Syed 

In [175]:
import joblib

tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
classifier = joblib.load('email_classifier.pkl')

predictions = classifier.predict(tfidf_vectorizer.transform(unlabeled_data['combined_text']))
print(len(predictions))
print(predictions)
unlabeled_data['predictions'] = predictions

KeyError: 'combined_text'

In [176]:
incorrect_index_list = []
for i in incorrect_index_list:
    correct_prediction(i, False)

unlabeled_data.to_csv("new_sample_set_with_predictions.csv", index=False)

# print(unlabeled_data['combined_text'], unlabeled_data['predictions']) 
# print(unlabeled_data.head())
i = 0
for text, prediction in zip(unlabeled_data['email body'], unlabeled_data['predictions']):
    print(f"Index: {unlabeled_data.index[i]} Text: {text}\nPrediction: {prediction}\n")
    i+=1

KeyError: 'predictions'

In [ ]:
unlabeled_data.at[1012, 'predictions'] = 0

In [ ]:
def mapPredictionsToIndices():
    index_prediction_map = {}
    for i, pred in enumerate(unlabeled_data['predictions']):
        if pred not in index_prediction_map:
            index_prediction_map[pred] = []
        index_prediction_map[pred].append(unlabeled_data.index[i])
    
    return index_prediction_map

In [ ]:
index_prediction_map = mapPredictionsToIndices()

for prediction, indices in index_prediction_map.items():
    print(f"Prediction: {prediction}, Indices: {indices}")


Prediction: 0, Indices: [180, 181, 182, 185, 186, 188, 189, 190, 192, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 208, 209, 212, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 241, 242, 253, 254, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 360, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 404, 413, 425, 440, 444, 445, 465, 479, 657, 663, 677, 793, 795, 796, 801, 804]
Prediction: 1, Indices: [183, 184, 187, 193, 194, 195, 196, 207, 210, 211, 213, 215, 239, 240, 243, 

In [177]:
# Train on labeled data
X_labeled = labeled_data['combined_text']
y_labeled = labeled_data['binary classification (expected output)']

tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))
X_labeled_tfidf = tfidf_vectorizer.fit_transform(X_labeled)

classifier = LogisticRegression()
classifier.fit(X_labeled_tfidf, y_labeled)

KeyError: 'combined_text'

In [179]:
# Predict probabilities on unlabeled data
X_unlabeled = unlabeled_data['combined_text']
try:
    print("Unlabeled data is not empty")
    X_unlabeled_tfidf = tfidf_vectorizer.transform(X_unlabeled)
except:
    print("Unlabeled data is empty")

KeyError: 'combined_text'

In [178]:
probabilities = classifier.predict_proba(X_unlabeled_tfidf)
uncertainty = 1 - probabilities.max(axis=1)  # Higher uncertainty = lower confidence
unlabeled_data['uncertainty'] = uncertainty

# Sort by uncertainty
most_uncertain = unlabeled_data.sort_values(by='uncertainty', ascending=False)
uncertain_num = len(unlabeled_data)
most_uncertain_subset = most_uncertain.index[:uncertain_num]
most_uncertain[['email subject', 'email body', 'uncertainty']].index[:uncertain_num]
# print the email body of the uncertain_num most uncertain emails
for i in range(uncertain_num):
    print(most_uncertain.index[i], most_uncertain['uncertainty'].iloc[i], most_uncertain['email body'].iloc[i])


ValueError: Length of values (631) does not match length of index (905)

In [ ]:
indices = []
indices = index_prediction_map[1]
# I want to label a set number of emails as job applications which ill do with the index
# of the most uncertain emails
# get the index of the uncertain_num most uncertain emails
# index = most_uncertain.index[:uncertain_num]
# print(index)
# label the emails as job applications
data.loc[indices, 'binary classification (expected output)'] = 1.0
# check if the emails have been labeled
print(data.loc[indices, 'binary classification (expected output)'])

183    1.0
184    1.0
187    1.0
193    1.0
194    1.0
      ... 
828    1.0
829    1.0
830    1.0
831    1.0
832    1.0
Name: binary classification (expected output), Length: 456, dtype: float64


In [ ]:
import numpy as np
# label the other indices in the subset as not job applications
not_indices = [val for val in most_uncertain_subset if val not in indices]
data.loc[not_indices, 'binary classification (expected output)'] = 0.0
print(data.loc[not_indices, 'binary classification (expected output)'])

206    0.0
268    0.0
242    0.0
404    0.0
329    0.0
      ... 
280    0.0
235    0.0
793    0.0
262    0.0
185    0.0
Name: binary classification (expected output), Length: 175, dtype: float64


In [ ]:
unlabeled_data = data[data['binary classification (expected output)'].isnull()]
after_unlabeled_data = len(unlabeled_data)
print(after_unlabeled_data)
labeled_data = data[data['binary classification (expected output)'].notnull()]
after_labeled_data = len(labeled_data)
print(after_labeled_data)

print(after_labeled_data - before_labeled_data)
print(after_unlabeled_data - before_unlabeled_data)

0
806
631
-631


In [ ]:
# Define your features (X) and labels (y)
X = data['combined_text']
y = data['binary classification (expected output)']  # This should be a column with binary labels (1 for job applications, 0 otherwise)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))

# Fit and transform the training data, transform the test data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train the classifier
classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

save_accuracy_score(accuracy)


Accuracy: 0.9629629629629629
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.91      0.95        57
           1       0.95      0.99      0.97       105

    accuracy                           0.96       162
   macro avg       0.97      0.95      0.96       162
weighted avg       0.96      0.96      0.96       162



In [ ]:
import joblib

# Save the vectorizer and model
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(classifier, 'email_classifier.pkl')


['email_classifier.pkl']

In [ ]:
# Load vectorizer and model
loaded_vectorizer = joblib.load('tfidf_vectorizer.pkl')
loaded_model = joblib.load('email_classifier.pkl')

# Preprocess and vectorize the new email
new_email = "This is a job application for the software engineer position."
new_email_tfidf = loaded_vectorizer.transform([new_email])

# Predict
prediction = loaded_model.predict(new_email_tfidf)
print("Prediction:", "Job Application" if prediction[0] == 1 else "Not a Job Application")

Prediction: Job Application


In [ ]:
!python --version
!pip show scikit-learn joblib
# code to get the numpy version?
import numpy as np
print(np.__version__)


Python 3.11.5
Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /Users/ary/anaconda3/lib/python3.11/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: imbalanced-learn
---
Name: joblib
Version: 1.2.0
Summary: Lightweight pipelining with Python functions
Home-page: https://joblib.readthedocs.io
Author: Gael Varoquaux
Author-email: gael.varoquaux@normalesup.org
License: BSD
Location: /Users/ary/anaconda3/lib/python3.11/site-packages
Requires: 
Required-by: imbalanced-learn, nltk, scikit-learn
1.26.3
